Find nearby Scheduled Monuments in England by postcode.

This code will find the nearest 5 scheduled monuments to the centre point of any given English postcode (just the first half!).


In [169]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
from shapely.geometry import Point
pd.set_option('display.max_colwidth', 175)
import folium

In [172]:
 """
    this function does awesome stuff
    
    Args:
        pcode: First half of any English postcode
        
    Returns:
        dunno- closest monuments maybe?
    """
def monument_finder(pcode): 
    monuments = gpd.read_file('Scheduled Monuments\\Monuments2.shp')
    monuments.to_crs(epsg=27700,inplace=True)
    postcodes = pd.read_csv('englishpostcodes3.csv')
    postcodes['eastings'] = pd.to_numeric(postcodes['eastings'],errors='coerce')
    postcodes['northings'] = pd.to_numeric(postcodes['northings'],errors='coerce')
    postcodes.dropna(inplace=True)
    geometry = [Point(xy) for xy in zip(postcodes['eastings'], postcodes['northings'])]
    postcodes['geometry'] = geometry
    postcodes = gpd.GeoDataFrame(postcodes)
    postcodes.set_crs(epsg=27700, inplace=True)
    postcode = postcodes.loc[postcodes['postcode'] == pcode, 'geometry'].values[0]
    monuments['distance'] = monuments['geometry'].distance(postcode) 
    monuments.sort_values(by='distance', ascending=True, inplace=True)
    monuments.reset_index(inplace=True)
   
    print(monuments['Name'].head())
    return monuments.head()

In [173]:
pcode='MK3'
closest_monuments=monument_finder(pcode)

C:\Users\inkyt\anaconda3\envs\egm722\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


0                                                                 Fishpond in Water Spinney 600m SE of St Giles's Church Tattenhoe
1                                  Moated site, fishponds and deserted medieval village of Tattenhoe, 300m west of Home Park Farm.
2    Moated site with associated earthworks, 770m south of St Mary's Church, south of the junction of Childs Way and Fulmer Street
3                                                                                         Roman town of Magiovinium and Roman fort
4                                                                                                        Medieval manor of Simpson
Name: Name, dtype: object


In [160]:

    #heres the code for putting the polygons of closest_monuments in the map
for _, r in closest_monuments.to_crs(epsg=4326).iterrows():
    # (code from the tutorial)Without simplifying the representation of each polygon,
    # the map might not be displayed
       # sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.01)
        # #eo_j = folium.GeoJson(data=geo_j,
                           #style_function=lambda x: {'fillColor': 'orange'})
    #folium.Popup(r['Name']).add_to(geo_j)
   # geo_j.add_to(my_map)
        centroid = gpd.GeoSeries(r['geometry']).set_crs(epsg=4326).to_crs(epsg=27700).centroid
        centroid = gpd.GeoSeries(centroid).set_crs(epsg=27700).to_crs(epsg=4326)
    #adding the centroid to the map,and ensuring it displays at the right scale
        folium.Marker(location=[centroid.y, centroid.x], popup= r['Name'],zoom_start=6).add_to(my_map)
# Project to [insert right crs)] projected crs
        closest_monuments = closest_monuments.to_crs(epsg=2263)
    # Access the centroid attribute of each polygon
        closest_monuments['centroid'] = closest_monuments.centroid
    # geometry (active) column
        closest_monuments = closest_monuments.to_crs(epsg=4326)

    # Centroid column
        closest_monuments['centroid'] = closest_monuments['centroid'].to_crs(epsg=4326)

        closest_monuments.head()

#my_map

   

In [161]:
my_map


In [154]:
my_map=folium.Map()